In [1]:
# Basic imports
import os
import getpass

# Langchain imports
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

# Tool imports
from langchain.tools import StructuredTool
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_experimental.utilities import PythonREPL

# Local imports
from thesis2024.models.coding_agent import CodingMultiAgent


def init_llm_langsmith(llm_key = 3, temp = 0.5):
    """Initialize the LLM model and LangSmith tracing."""
    # Set environment variables
    def _set_if_undefined(var: str):
        if not os.environ.get(var):
            os.environ[var] = getpass(f"Please provide your {var}")
    _set_if_undefined("OPENAI_API_KEY")
    _set_if_undefined("LANGCHAIN_API_KEY")

    # Add tracing in LangSmith.
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    if llm_key == 3:
        llm_ver = "gpt-3.5-turbo-0125"
        os.environ["LANGCHAIN_PROJECT"] = "GPT-3.5 Teaching Agent System TEST 1"
    elif llm_key == 4:
        llm_ver = "gpt-4-0125-preview"
        os.environ["LANGCHAIN_PROJECT"] = "GPT-4 Teaching Agent System TEST 1"

    llm_model = ChatOpenAI(model_name=llm_ver, temperature=temp)
    return llm_model

llm_model = init_llm_langsmith(llm_key=3, temp=0)
tools = []

## Define Tools

In [24]:
"""Search tool."""
search = DuckDuckGoSearchAPIWrapper()
search_tool = Tool(name="Current Search",
                   func=search.run,
                   description="Useful when you need to answer questions about nouns, current events or the current state of the world."
                   )

tools.append(search_tool)

In [ ]:
"""Coding multi-agent tool."""
coding_subgraph_class = CodingMultiAgent(llm_model=llm_model)
coding_graph = coding_subgraph_class.instanciate_graph()


def coding_function(query: str):
    """Coding tool function."""
    output = coding_graph.invoke({"messages": [HumanMessage(content=query)]},
                        {"recursion_limit": 100})
    return output["messages"][-1].content


coding_tool = StructuredTool.from_function(
                            func=coding_function,
                            name="Coding Tool",
                            description="Useful when you need to answer questions using a coded example."
                            )

tools.append(coding_tool)

# Running Teaching Agents System with Memory

In [2]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
SYSTEM_MESSAGE = """You will interact with a student who has no prior knowledge of the subject."""
COURSE = """Introduction to Computer Science"""
SUBJECT = """Gradient Descent"""

prompt_hub_template = hub.pull("augustsemrau/react-teaching-chat").template
prompt_template = PromptTemplate.from_template(template=prompt_hub_template)
prompt = prompt_template.partial(system_message=SYSTEM_MESSAGE, course_name=COURSE, subject_name=SUBJECT)
print(prompt)

agent = create_react_agent(llm=llm_model, tools=tools, prompt=prompt, output_parser=None)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True)


input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] partial_variables={'system_message': 'You will interact with a student who has no prior knowledge of the subject.', 'course_name': 'Introduction to Computer Science', 'subject_name': 'Gradient Descent'} template='Teaching Assistant is a large language model trained by OpenAI.\n\nTeaching Assistant is designed to be able to assist with teaching related tasks, from answering simple questions to providing in-depth explanations and discussions on STEM topics. \nAs a language model, Teaching Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nTeaching Assistant is constantly learning and improving, and its capabilities are constantly evolving. \nIt is able to process and understand large amounts of text, and can use this knowledge to provide accurate

In [3]:
agent_executor.invoke({"input": "Hello I am August?",})["output"]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hello August! How can I assist you today with learning about Gradient Descent in Computer Science?

> Finished chain.


'Hello August! How can I assist you today with learning about Gradient Descent in Computer Science?'

In [4]:
agent_executor.invoke({"input": "Can you look up what it is about?",})#["output"]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search for the definition of Gradient Descent
Action Input: Look up the definition of Gradient DescentSearch for the definition of Gradient Descent is not a valid tool, try one of [].Do I need to use a tool? No
Final Answer: Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving in the direction of the steepest descent. It is commonly used in training neural networks to update the weights and biases to improve the model's performance.

> Finished chain.


{'input': 'Can you look up what it is about?',
 'chat_history': [HumanMessage(content='Hello I am August?'),
  AIMessage(content='Hello August! How can I assist you today with learning about Gradient Descent in Computer Science?'),
  HumanMessage(content='Can you look up what it is about?'),
  AIMessage(content="Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving in the direction of the steepest descent. It is commonly used in training neural networks to update the weights and biases to improve the model's performance.")],
 'output': "Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving in the direction of the steepest descent. It is commonly used in training neural networks to update the weights and biases to improve the model's performance."}

In [5]:
agent_executor.invoke({"input": "Please code an example of gradient descent for me.",})#["output"]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Sure! In coding an example of gradient descent, we typically start by defining a loss function and then iteratively update the parameters (weights and biases) of the model to minimize this loss function. This process involves calculating the gradient of the loss function with respect to the parameters and moving in the opposite direction of this gradient to reach the minimum. Would you like me to provide a simple example in Python?

> Finished chain.


{'input': 'Please code an example of gradient descent for me.',
 'chat_history': [HumanMessage(content='Hello I am August?'),
  AIMessage(content='Hello August! How can I assist you today with learning about Gradient Descent in Computer Science?'),
  HumanMessage(content='Can you look up what it is about?'),
  AIMessage(content="Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving in the direction of the steepest descent. It is commonly used in training neural networks to update the weights and biases to improve the model's performance."),
  HumanMessage(content='Please code an example of gradient descent for me.'),
  AIMessage(content='Sure! In coding an example of gradient descent, we typically start by defining a loss function and then iteratively update the parameters (weights and biases) of the model to minimize this loss function. This process involves calculating the gradient of the loss func

In [7]:
agent_executor.invoke({"input": "Yes.",})#["output"]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Explain the concept of Gradient Descent
Action Input: Provide a detailed explanation of Gradient Descent to the studentExplain the concept of Gradient Descent is not a valid tool, try one of [].Do I need to use a tool? No
Final Answer: Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving in the direction of the steepest descent. It is commonly used in training neural networks to update the weights and biases to improve the model's performance. In coding an example of gradient descent, we typically start by defining a loss function and then iteratively update the parameters (weights and biases) of the model to minimize this loss function. This process involves calculating the gradient of the loss function with respect to the parameters and moving in the opposite direction of this gradient to reach the minimum.

> Fi

{'input': 'Yes.',
 'chat_history': [HumanMessage(content='Hello I am August?'),
  AIMessage(content='Hello August! How can I assist you today with learning about Gradient Descent in Computer Science?'),
  HumanMessage(content='Can you look up what it is about?'),
  AIMessage(content="Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving in the direction of the steepest descent. It is commonly used in training neural networks to update the weights and biases to improve the model's performance."),
  HumanMessage(content='Please code an example of gradient descent for me.'),
  AIMessage(content='Sure! In coding an example of gradient descent, we typically start by defining a loss function and then iteratively update the parameters (weights and biases) of the model to minimize this loss function. This process involves calculating the gradient of the loss function with respect to the parameters and moving

# Leftover Code


In [ ]:
# search = DuckDuckGoSearchAPIWrapper()

# search_tool = Tool(name="Current Search",
#                    func=search.run,
#                    description="Useful when you need to answer questions about nouns, current events or the current state of the world."
#                    )

# tools = [search_tool]

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# prompt = ChatPromptTemplate.from_messages(
#     [
#         SystemMessage(
#             content="You are a chatbot teaching assistant having a conversation with a human."),
#             # The persistent system prompt
#         MessagesPlaceholder(
#             variable_name="chat_history"),
#             # Where the memory will be stored.
#         HumanMessagePromptTemplate.from_template(
#             "{human_input}"),
#             # Where the human input will injected
#     ]
# )

# llm = ChatOpenAI()

# chat_llm_chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     verbose=True,
#     memory=memory,
# )

# chat_llm_chain.predict(human_input="Hi there my name is August Semrau.")

In [ ]:
# # This defines the object that is passed between each node
# # in the graph. We will create different nodes for each agent and tool
# class AgentState(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], operator.add]
#     sender: str

# def agent_node(self, state, agent, name):
#     """Helper function to create a node for a given agent. Node that invokes agent."""
#     result = agent.invoke(state)
#     # We convert the agent output into a format that is suitable to append to the global state
#     if isinstance(result, FunctionMessage):
#         pass
#     else:
#         result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
#     return {
#         "messages": [result],
#         # Since we have a strict workflow, we can
#         # track the sender so we know who to pass to next.
#         "sender": name,
#     }

# def create_frontline_node(self, state: BaseMessage):
#     """Create the frontline node."""
#     system_message = """You are frontline teaching assistant.
#                     Be concise in your responses.
#                     You can chat with students and help them with basic questions, but if the student has a teaching-related question,
#                     do not try to answer the question directly or gather information.
#                     Instead, immediately transfer them to the teaching agent system by asking the user to hold for a moment.
#                     Otherwise, just respond conversationally."""
#     prompt = ChatPromptTemplate.from_messages([("system", system_message),
#                                                 MessagesPlaceholder(variable_name="messages")])
#     chain = prompt | self.model
#     node = functools.partial(self.agent_node, agent=chain, name="Frontline")
#     return node

# """Teaching Agent System Graph."""
# graph = StateGraph(AgentState)

# frontline_node = create_frontline_node()
# graph.add_node(frontline_node)
# graph.set_entry_point(frontline_node)



